In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from plotting import newfig, savefig
from torch.optim.lr_scheduler import StepLR
from IPython.display import display, clear_output
class neural_net(nn.Module):
    def __init__(self, NL, NN):
        self.NL = NL
        self.NN = NN
        super(neural_net, self).__init__()
        self.input_layer = nn.Linear(1, NL)
        self.hidden_layer = nn.Linear(NN, int(NN/2))
        self.output_layer = nn.Linear(int(NN/2), 1)


    def forward(self,x):
        out = torch.tanh(self.input_layer(x))
        out = torch.tanh(self.hidden_layer(out))
        out_final = self.output_layer(out)
        return out_final

In [ ]:
class PINN(nn.Module):
    def __init__(self, x):
        super(PINN, self).__init__()
        self.x = x
        N = 20000
        net = neural_net(4, 20)
        self.mse = nn.MSELoss(reduction='mean')
        self.optimizer = optim.Adam(net.parameters(), lr=1e-4)
        x_0 = torch.zeros(2000, 1)
        y_0 = net(self.x_0)


    def ode(self):
        y = self.net(self.x)
        y_x = torch.autograd.grad(y, self.x, grad_outputs=torch.ones_like(self.net(self.x)), create_graph=True)[0]
        return y-y_x


    def loss_function(self):
        y = self.net(self.x)
        y_x = torch.autograd.grad(y, self.x, grad_outputs=torch.ones_like(self.net(self.x)), create_graph=True)[0]
        y_xx = torch.autograd.grad(y_x, self.x, grad_outputs=torch.ones_like(self.net(self.x)), create_graph=True)[0]
        return self.mse(y_xx, -self.ode())

    def train(self, epochs=10000):
        self.net.train()
        self.optimizer.zero_grad()
        loss = self.loss_function(self.x, self.net)
        loss.backward()
        self.optimizer.step()
        return loss.item()

